In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import joblib

# Carregar dados processados
df = pd.read_csv('dados_hipertensao_processados.csv')

# 1. Preparar dados para modelo
features = ['idade', 'imc', 'pressao_sistolica', 'pressao_diastolica', 'frequencia_cardiaca']
X = df[features]
y = df['complicacao']

# 2. Tratar valores missing
X = X.fillna(X.mean())

# 3. Normalizar dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. Dividir dados - usando stratify para manter proporção de classes
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y,
    test_size=0.3,
    random_state=42,
    stratify=y  # importante para conjuntos pequenos
)

# 5. Treinar modelo
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced'  # ajuda com conjuntos desbalanceados
)
model.fit(X_train, y_train)

# 6. Avaliar modelo
y_pred = model.predict(X_test)
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))
print(f"\nAcurácia: {accuracy_score(y_test, y_pred):.2%}")

# 7. Salvar modelo e scaler
joblib.dump(model, 'modelo_risco_hipertensao.pkl')
joblib.dump(scaler, 'scaler.pkl')

# 8. Função para fazer previsões
def prever_risco(paciente):
    """
    paciente: lista com [idade, imc, pressao_sistolica, pressao_diastolica, frequencia_cardiaca]
    Retorna a probabilidade de complicação (0-1)
    """
    if len(paciente) != 5:
        raise ValueError("O paciente deve ter exatamente 5 características")

    paciente_scaled = scaler.transform([paciente])
    proba = model.predict_proba(paciente_scaled)[0][1]
    return proba

# 9. Exemplo de uso CORRETO (apenas 5 features)
paciente_exemplo = [50, 28, 145, 92, 85]  # idade, imc, sistólica, diastólica, FC
try:
    risco = prever_risco(paciente_exemplo)
    print(f"\nExemplo - Risco de complicação: {risco:.1%}")
except ValueError as e:
    print(f"Erro: {e}")

# 10. Análise de importância das features
print("\nImportância das características:")
for feature, importance in zip(features, model.feature_importances_):
    print(f"{feature}: {importance:.2%}")


Relatório de Classificação:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         2
         1.0       1.00      1.00      1.00         1

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3


Acurácia: 100.00%

Exemplo - Risco de complicação: 68.0%

Importância das características:
idade: 16.34%
imc: 26.14%
pressao_sistolica: 31.83%
pressao_diastolica: 10.99%
frequencia_cardiaca: 14.71%


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
